### Public Private Key generation

In [1]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa

# Generate a new RSA key pair
private_key = rsa.generate_private_key(
    public_exponent=65537,  # Common choice for RSA
    key_size=2048,  # Key size in bits
    backend=default_backend()
)

# Serialize and save the private key to a file
with open("private_key.pem", "wb") as private_key_file:
    private_key_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    private_key_file.write(private_key_pem)

# Extract and serialize the public key
public_key = private_key.public_key()
with open("public_key.pem", "wb") as public_key_file:
    public_key_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    public_key_file.write(public_key_pem)

print("Keys generated and saved.")


Keys generated and saved.


### Sign a verifiable credential using a private key. 

In [14]:
import json
import hashlib
import base64
from datetime import datetime
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

# Example verifiable credential data
credential_data = { "credential": {
    "@context": ["https://www.w3.org/2018/credentials/v1"],
    "type": ["VerifiableCredential"],
    "issuer": "did:example:123456789abcdefghi",
    "issuanceDate": datetime.utcnow().isoformat(),
    "credentialSubject": {
        "id": "did:example:123456789abcdefg",
        "degree": {
            "type": "BachelorDegree",
            "name": "Computer Science",
            "university": "Alice's University",
            "graduationDate": "2023-06-15"
        }
    }}
}    

# Load your private key (you should manage this securely)
with open("private_key.pem", "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,
        backend=default_backend()
    )

# Convert the credential data to JSON string
credential_json = json.dumps(credential_data["credential"], separators=(',', ':'), sort_keys=True)

# Calculate the SHA-256 hash of the credential JSON
hash_algorithm = hashes.SHA256()
hasher = hashlib.sha256()
hasher.update(credential_json.encode('utf-8'))
hash_value = hasher.digest()

# Sign the hash value using the private key
signature = private_key.sign(
    hash_value,
    padding.PKCS1v15(),
    hashes.SHA256()
)

# Convert the signature to base64
base64_signature = base64.b64encode(signature).decode('utf-8')

# Add the signature to the credential data
credential_data["proof"] = {
    "type": "RsaSignature2018",
    "created": datetime.utcnow().isoformat(),
    "proofPurpose": "assertionMethod",
    "verificationMethod": "did:example:123456789abcdefghi#keys-1",
    "signatureValue": base64_signature
}

# Print the signed credential JSON
print(json.dumps(credential_data, indent=4))


{
    "credential": {
        "@context": [
            "https://www.w3.org/2018/credentials/v1"
        ],
        "type": [
            "VerifiableCredential"
        ],
        "issuer": "did:example:123456789abcdefghi",
        "issuanceDate": "2023-08-28T10:54:48.360814",
        "credentialSubject": {
            "id": "did:example:123456789abcdefg",
            "degree": {
                "type": "BachelorDegree",
                "name": "Computer Science",
                "university": "Alice's University",
                "graduationDate": "2023-06-15"
            }
        }
    },
    "proof": {
        "type": "RsaSignature2018",
        "created": "2023-08-28T10:54:48.409242",
        "proofPurpose": "assertionMethod",
        "verificationMethod": "did:example:123456789abcdefghi#keys-1",
        "signatureValue": "oh76pRMS+FpDwrWNrwm0tlHqiYbeiwTURzMyNxinmKvXUu5h8ru+XxIE4Fs5zPt/6t4hFJOLdDjepRisXNYzGpzH8RrcGkcNR8kWmWwSOc3pFLvGUxqLGlc1dMpVMDOl7G/Er3XqiwY2lBav/+t1nSfvFjEDO+2e

### Encode the JSON-LD representation of the signed credential into a QR code.

In [15]:
import json
import qrcode

# Assuming credential_data is the JSON-LD representation of the signed credential

# Convert the credential data to JSON string
credential_json = json.dumps(credential_data)

# Create a QR code instance
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)

# Add the JSON data to the QR code
qr.add_data(credential_json)
qr.make(fit=True)

# Create an image from the QR code
qr_image = qr.make_image(fill_color="black", back_color="white")

# Save the QR code image
qr_image.save("vc_qr_code.png")


### Verify a signed json using same public key

In [16]:
credential_data

{'credential': {'@context': ['https://www.w3.org/2018/credentials/v1'],
  'type': ['VerifiableCredential'],
  'issuer': 'did:example:123456789abcdefghi',
  'issuanceDate': '2023-08-28T10:54:48.360814',
  'credentialSubject': {'id': 'did:example:123456789abcdefg',
   'degree': {'type': 'BachelorDegree',
    'name': 'Computer Science',
    'university': "Alice's University",
    'graduationDate': '2023-06-15'}}},
 'proof': {'type': 'RsaSignature2018',
  'created': '2023-08-28T10:54:48.409242',
  'proofPurpose': 'assertionMethod',
  'verificationMethod': 'did:example:123456789abcdefghi#keys-1',
  'signatureValue': 'oh76pRMS+FpDwrWNrwm0tlHqiYbeiwTURzMyNxinmKvXUu5h8ru+XxIE4Fs5zPt/6t4hFJOLdDjepRisXNYzGpzH8RrcGkcNR8kWmWwSOc3pFLvGUxqLGlc1dMpVMDOl7G/Er3XqiwY2lBav/+t1nSfvFjEDO+2e9TAfbI2W2VkfmJTDjvEan2+RJeEhUFBf7Ga4GlWC+VrKEyo8jNw2vlvkudj7KyxYdHRDfwsSa3865Oh/RRpfNmt3P2HU/ctIYfzoxbgj9svMs/UjTS0hcWBGjijMwMuF2T9qewJCAVpcAsEI0yrVWzD8G3kmUC1QPTpommXo7rxc5KLmTFnXJg=='}}

In [18]:
import json
import base64
import hashlib
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

signed_vc_data = credential_data

# Extract necessary data from the signed VC
signature_value = base64.b64decode(signed_vc_data["proof"]["signatureValue"])

# Load the public key from the public key PEM file
with open("public_key.pem", "rb") as public_key_file:
    public_key = serialization.load_pem_public_key(
        public_key_file.read(),
        backend=default_backend()
    )

# Extract the credential JSON for hashing
credential_json = json.dumps(signed_vc_data["credential"], separators=(',', ':'), sort_keys=True)

# Calculate the SHA-256 hash of the credential JSON
hash_algorithm = hashes.SHA256()
hasher = hashlib.sha256()
hasher.update(credential_json.encode('utf-8'))
hash_value = hasher.digest()

credential_json
# Verify the signature
try:
    public_key.verify(
        signature_value,
        hash_value,
        padding.PKCS1v15(),
        hash_algorithm
    )
    print("Verification successful: VC is authentic.")
except Exception as e:
    print("Verification failed:", e.message)


Verification successful: VC is authentic.


### Sample example of public/private key signing

In [13]:
import base64
import hashlib
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

# Generate a new RSA key pair (for demonstration purposes)
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)

# Serialize and save the private key (for demonstration purposes)
with open("private_key.pem", "wb") as private_key_file:
    private_key_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        encryption_algorithm=serialization.NoEncryption()
    )
    private_key_file.write(private_key_pem)

# Extract the public key from the private key
public_key = private_key.public_key()

# Serialize and save the public key (for demonstration purposes)
with open("public_key.pem", "wb") as public_key_file:
    public_key_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    public_key_file.write(public_key_pem)

# Message to be signed and verified
message = b"This is a test message."

# Sign the message using the private key
signature = private_key.sign(
    message,
    padding.PKCS1v15(),
    hashes.SHA256()
)

# Verify the signature using the public key
try:
    public_key.verify(
        signature,
        message,
        padding.PKCS1v15(),
        hashes.SHA256()
    )
    print("Verification successful: Signature is valid.")
except Exception as e:
    print("Verification failed:", e)


Verification successful: Signature is valid.
